In [8]:
# wir benötigen wieder ein paar python imports...
import sys
from os import environ
import os
import uuid
sys.path.append(os.path.abspath(".."))

from sparql_queries import *
from connector_client import ConnectorClient
from connector.messages.datamodel_utils import SubscriptionRegisterRequest, SubscriptionUnregisterRequest # generic messages within the connector_base repo

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

In [9]:
# graphdb queries, um den richtigen connector zu identifizieren
await client.load_connector_config()
await client.return_connectors()

dict_keys(['milling_station', 'injection_molding', 'assembly_station'])

In [10]:
# wechsel an den ausgewählten, richtigen connector
await client.switch_connector('milling_station')

In [11]:
# subscription_id aus dem Knowledge Graph "ziehen"
output = await client.query_graphdb(build_subscription_query())
print(output)

                           subscription                                device  \
0  b545fd3f-833d-43d1-931c-8e78566b580f  44ae86d9-6f53-4e92-8714-60d5b18d5f36   

                              datapoint                       start   state  
0  eabbec0d-a147-411c-bc74-aafa2042de1f  2025-04-28T20:06:55.066871  Active  


In [12]:
subscription_id = output.loc[0, "subscription"]
# definieren einer Unsubscribe-Nachricht für einen OPC-UA-Server
example_unsubscribe_payload = SubscriptionUnregisterRequest(
    subscription_identifier = subscription_id,
    device_origin="client"
)

In [13]:
await client.unsubscribe_data_source(base_payload = example_unsubscribe_payload)

message type: <class 'connector.messages.datamodel_utils.SubscriptionClosedResponse'>
Matching response received:
{
    "type": "SubscriptionClosedResponse",
    "device_origin": "module_opcua",
    "subscription_identifier": "b545fd3f-833d-43d1-931c-8e78566b580f",
    "status": "success"
}


MsgModel(root=ResponseMessage(version='1.0.0', payload=CommandResponse(type='CommandResponse', base_payload=SubscriptionClosedResponse(type='SubscriptionClosedResponse', device_origin='module_opcua', subscription_identifier=UUID('b545fd3f-833d-43d1-931c-8e78566b580f'), status='success')), type='ResponseMessage', message_id=UUID('3443664d-712e-4ba7-829a-60db3a04eefe'), correlation_id=UUID('0d96162a-2c7f-44c4-9ca9-dbea1a71283d'), timestamp=datetime.datetime(2025, 4, 28, 20, 7, 13, 681812), status_code='Valid', service_id='milling_station'))